In [3]:
# Import Libraries
import numpy as np
import pandas as pd
from keras.optimizers import *
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedShuffleSplit
from IPython.display import HTML, display
import tabulate
from keras.layers import Dense,Conv1D,Dropout,MaxPooling1D,Flatten,Reshape
from google.colab import drive

Using TensorFlow backend.


In [4]:
# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!ls "/content/drive/My Drive"

 170145._1996_Parte4.pdf
'1 Congreso de Ingeniería de Sistemas e Informática, Bucaramanga.docx'
 2015-03-27-EISI-16-18-25.JPG
 20160824_151439.jpg
 20160824_152302.jpg
 20161213_204702.jpg
 20170110_201228.jpg
 20170301_074401.jpg
 20170301_074408.jpg
 20170319_171357.jpg
 29_Demo.apk
 acta.pdf
'Acuerdo 24'
 Amor
'Analisis numérico'
'Anthony Robbins - Despertando al gigante interior-FREELIBROS.COM.pdf'
 app-release.apk
'App Web Experimental'
 Artículo
'Artículo Traducido.gdoc'
 audio-acta.zip
'Audios Tutoriales'
 base.backup
 baseheansesoft.backup
'Bases de datos'
 Cardales_1N_Final.xlsx
'Carpeta compartida con Amorcitu'
'Carta de Motivación.gdoc'
 Casabe_1060_Final_SS.xlsx
 Casabe_1069_Final_SS_2.xlsx
'CertificaciónLicencia InHousev2.pdf'
 Certificación.pdf
'Cine economía y sociedad'
 clase12032019.gdoc
'Colab Notebooks'
'Compartida con Jeyson'
'Compartido con Miguel'
'Compartido Hermana'
 com.wolfram.android.alpha-1.3.0.4590401-paid-8cbf-www.apkhere.com.apk
'CONISI 2016'
'CO

In [0]:
#Dataset
dataset = pd.read_csv('/content/drive/My Drive/Keras_DNN/Dataset/data_windows_132.csv').iloc[:,1:]

In [7]:
dataset.shape

(29848, 133)

In [0]:
#Delete NAN values
"""
dataset_x = []
dataset_y = []
for label in range(1,17):
    data = dataset[dataset.iloc[:,0] == label]
    for index, row in data.iterrows():
        if (len(dataset_x) <= (26*label)):
            dataset_x.append(np.nan_to_num(dataset.iloc[index,1:].values))
            dataset_y.append(dataset.iloc[index,0:1].values)
        else:
            break
"""
dataset_x = []
dataset_y = []

count = np.zeros(17)

for label in range(1,17):
    data = dataset[dataset.iloc[:,0] == label]
    for index, row in data.iterrows():
        if (len(dataset_x) <= (840*label)):
          dataset_x.append(np.nan_to_num(dataset.iloc[index,1:].values))
          dataset_y.append(dataset.iloc[index,0:1].values)
          count[label]+=1
    
dataset_x = np.array(dataset_x)
dataset_y = np.array(dataset_y)

In [9]:
dataset_x.shape,dataset_y.shape

((13441, 132), (13441, 1))

In [10]:
print(min(count[1:]))
print(count[1:])

840.0
[841. 840. 840. 840. 840. 840. 840. 840. 840. 840. 840. 840. 840. 840.
 840. 840.]


In [0]:
#Normalice X values
mean = np.sum(dataset_x,axis=0,keepdims=True)/dataset_x.shape[0]
normal_variance = np.sum(dataset_x**2,axis=0,keepdims=True)/dataset_x.shape[0]
dataset_x_normalice = dataset_x-mean
dataset_x_normalice = dataset_x_normalice/np.sqrt(normal_variance+1e-8)

In [12]:
dataset_y_reshape = dataset_y.reshape(1,dataset_y.shape[0]).flatten().astype(int)
dataset_y_reshape[20:40]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [13]:
#Create Y binary class vector
y_train_vector = dataset_y_reshape
print(y_train_vector[0])
print(y_train_vector.shape)

1
(13441,)


In [0]:
#Dividimos en train y test
#kf = KFold(n_splits=4)
sss =  StratifiedShuffleSplit(n_splits=4,test_size=0.2, random_state=1)
#kf.get_n_splits(dataset_x_normalice)
sss.get_n_splits(dataset_x_normalice, y_train_vector)
index_kfol = []
for train_index, test_index in sss.split(dataset_x_normalice, y_train_vector):
    index_kfol.append([train_index,test_index])

In [19]:
train_index = index_kfol[0][0]
test_index = index_kfol[0][1]

print(train_index.shape)
print(test_index.shape)

(10752,)
(2689,)


In [0]:
def test_configuration(lr_v = [0.001], beta_1_v=[0.9],beta_2_v=[0.999] ,epochs_n = [10], batch_size_n = [32], input_shape_n = 132):
    
    configuration = 1
    table = [["Config","Train 1","Test 1","Train 2","Test 2","Train 3","Test 3","Train 4","Test 4","Train Prom","Test Prom"]]

    
    #Dividimos en train y test
    sss =  StratifiedShuffleSplit(n_splits=4,test_size=0.2, random_state=1)
    #kf.get_n_splits(dataset_x_normalice)
    sss.get_n_splits(dataset_x_normalice, y_train_vector)
    index_kfol = []
    for train_index, test_index in sss.split(dataset_x_normalice, y_train_vector):
      index_kfol.append([train_index,test_index])
                        
    #For save accuracies
    results = {}
                        
    # Create Model
    model = Sequential([
                        Reshape((12, 11), input_shape=(132,)),
                        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(12,11)),
                        Conv1D(filters=64, kernel_size=2, activation='relu'),
                        Dropout(0.5),
                        MaxPooling1D(pool_size=2),
                        Flatten(),                    
                        Dense(128, activation='relu'),
                        Dense(32, activation='relu'),
                        Dense(16, activation='softmax'),
                    ])
    
    model.compile(
                  optimizer="Adam",
                  loss='categorical_crossentropy',
                  metrics=['accuracy'],
                )
                                              
    Wsave = model.get_weights() 

    for batch in batch_size_n:
        for epoch in epochs_n:
            for beta2 in beta_2_v:
                for beta1 in beta_1_v:
                    for lr in lr_v:
                        #Compile Model
                        model.compile(
                          optimizer=Adam(lr=lr, beta_1=beta1, beta_2=beta2, amsgrad=False),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'],
                        )              

                        for g in range(4):

                            model.set_weights(Wsave)

                            #Get K-fold group 
                            train_index = index_kfol[g][0]
                            test_index = index_kfol[g][1]

                            x_train = dataset_x_normalice[train_index]
                            y_train = y_train_vector[train_index]
                            x_test = dataset_x_normalice[test_index]
                            y_test = y_train_vector[test_index]  

                            history = model.fit(
                              x_train, # training data
                              to_categorical(y_train-1), # training targets
                              epochs=epoch,
                              batch_size=batch,
                              verbose=0
                            )

                            results[str(g)+"train"] = history.history['acc'][-1]

                            output = model.evaluate(x_test,to_categorical(y_test-1))

                            results[str(g)+"test"] = output[1]
                            
                        #Print Results
                        print("______________________Configuration-"+str(configuration)+"______________________")
                        print("LR:"+str(lr)+"- Beta_1:"+str(beta1)+"- Beta_2:"+str(beta2))
                        print("Epochs:"+str(epoch)+"- Bach:"+str(batch))
                        print("_________________________________________________________")
                        
                        prom_train = []
                        prom_test = []
                        for g in range(4):
                            prom_train.append(results[str(g)+"train"]*100)
                            prom_test.append(results[str(g)+"test"]*100)
                        
                        table.append([configuration,
                                      "%.2f" % round(results["0train"]*100,2),"%.2f" % round(results["0test"]*100,2),
                                      "%.2f" % round(results["1train"]*100,2),"%.2f" % round(results["1test"]*100,2),
                                      "%.2f" % round(results["2train"]*100,2),"%.2f" % round(results["2test"]*100,2),
                                      "%.2f" % round(results["3train"]*100,2),"%.2f" % round(results["3test"]*100,2),
                                      "%.2f" % round(sum(prom_train)/4,2),"%.2f" % round(sum(prom_test)/4,2)])
                        print("_______________________ENDTEST_________________________")   
                        configuration = configuration+1
                        
    display(HTML(tabulate.tabulate(table, tablefmt='html')))       

In [21]:
lr_t=[0.00094]
beta_1_t = [0.9]
beta_2_t = [0.999]
epochs_t = [20]
batch_size_t = [32]
test_configuration(lr_v=lr_t,beta_1_v=beta_1_t,beta_2_v=beta_2_t,epochs_n = epochs_t,batch_size_n = batch_size_t)

2689/2689 [==============================] - 0s 44us/step
______________________Configuration-1______________________
LR:0.00094- Beta_1:0.9- Beta_2:0.999
Epochs:20- Bach:32
_________________________________________________________
_______________________ENDTEST_________________________


Config,Train 1,Test 1,Train 2,Test 2,Train 3,Test 3,Train 4,Test 4,Train Prom,Test Prom
1,97.25,96.84,97.45,96.99,97.49,96.17,97.48,96.50,97.42,96.63
